In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

img = cv2.imread('/content/drive/My Drive/ẢNH MRI CARGO/Y8.jpg', cv2.IMREAD_GRAYSCALE)

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
img2 = np.zeros((img.shape[0],img.shape[1]))

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img2[i][j] = img[i][j]

a = 0
for i in range(img.shape[0]//5,img.shape[0]-img.shape[0]//5):
    tong = 0
    dem = 0
    for j in range(img.shape[1]):
        if img2[i][j] >= 70:
            tong += img2[i][j]
            dem += 1
    if a < tong/dem:
        a = tong/dem
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if img2[i][j] < a:
            img2[i][j] = 0

plt.imshow(img2, cmap='gray')
plt.show()

In [ ]:
win = 2
#tính cường độ trung bình
intensity = np.zeros((img.shape[0]//win,img.shape[1]//win))
for i in range(intensity.shape[0]):
    for j in range(intensity.shape[1]):
        for m in range(win):
            for n in range(win):
                intensity[i][j] += img[win*i+m][win*j+n]
        intensity[i][j] = intensity[i][j] // (win**2)
        
#intensity matriX
inten_threshold = 17
degree_threshold = 3
inten_matrix = np.zeros((intensity.shape[0],intensity.shape[1]))
for i in range(intensity.shape[0]-1):
    for j in range(intensity.shape[1]-1):
        for m in range(-1,2):
            for n in range(-1,2):
                if abs(intensity[i][j] - intensity[i+m][j+n]) > inten_threshold:
                    inten_matrix[i][j] +=1
            
for i in range(intensity.shape[0]):
    for j in range(intensity.shape[1]):
        if inten_matrix[i][j] >= degree_threshold:
            inten_matrix[i][j] = 1
        else:
            inten_matrix[i][j] = 0

inten_matrix = cv2.resize(inten_matrix, (250,300))
plt.imshow(inten_matrix, cmap='gray')

In [ ]:
# tính entropy
win = 2
import pandas as pd
from scipy.stats import entropy as en
entropy = np.zeros((img.shape[0] - win+1,img.shape[1] - win+1))
for i in range(entropy.shape[0]):
    for j in range(entropy.shape[1]):
        count = np.zeros(win**2)
        for m in range(win):
            for n in range(win):
                count[m*win+n] = img[i+m][j+n]
        if max(count) - min(count) < 17:
            entropy[i][j] = 0
        else:
            entropy[i][j] = en(pd.Series(count).value_counts(), base=10)

#entropy matrix
entr_threshold = 0.1
entr_matrix = np.zeros((entropy.shape[0],entropy.shape[1]))
for i in range(entropy.shape[0]-1):
    for j in range(entropy.shape[1]-1):
        for m in range(-1,2):
            for n in range(-1,2):
                if abs(entropy[i][j] - entropy[i+m][j+n]) > entr_threshold:
                    entr_matrix[i][j] +=1

# degree = 3
# for i in range(entropy.shape[0]):
#     for j in range(entropy.shape[1]):
#         if entr_matrix[i][j] >= degree:
#             entr_matrix[i][j] = 1
#         else:
#             entr_matrix[i][j] = 0

plt.imshow(entr_matrix, cmap = 'gray')

In [ ]:
#joint matrix
new_inten_matrix = cv2.resize(inten_matrix, (250,300))
new_entr_matrix = cv2.resize(entr_matrix, (250,300))
new_img2 = cv2.resize(img2, (250,300))
joint_matrix = new_inten_matrix * new_entr_matrix * new_img2

for i in range(300):
    for j in range(250):
        if joint_matrix[i][j] > 0:
            joint_matrix[i][j] = 1
        else:
            joint_matrix[i][j] = 0

plt.imshow(joint_matrix, cmap = 'gray')